In [1]:
from __future__ import print_function

import numpy as np

from keras.callbacks import EarlyStopping
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
np.random.seed(1000)

In [3]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()


In [4]:
xgb = Sequential()

xgb.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
xgb.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
xgb.add(MaxPooling2D(pool_size=(2, 2)))
xgb.add(Dropout(0.25))
xgb.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
xgb.add(MaxPooling2D(pool_size=(2, 2)))
xgb.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
xgb.add(MaxPooling2D(pool_size=(2, 2)))
xgb.add(Dropout(0.25))
xgb.add(Flatten())
xgb.add(Dense(1024, activation='relu'))
xgb.add(Dropout(0.5))
xgb.add(Dense(10, activation='softmax'))

In [5]:
xgb.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.0001, decay=1e-6),
                  metrics=['accuracy'])


In [6]:
xgb.fit(X_train / 255.0, to_categorical(Y_train),
              batch_size=128,
              shuffle=True,
              epochs=250,
              validation_data=(X_test / 255.0, to_categorical(Y_test)),
              callbacks=[EarlyStopping(min_delta=0.001, patience=3)])

Train on 50000 samples, validate on 10000 samples
Epoch 1/250
50000/50000 [==============================] - 72s 1ms/step - loss: 1.9828 - accuracy: 0.2619 - val_loss: 1.7172 - val_accuracy: 0.3818
Epoch 2/250
50000/50000 [==============================] - 73s 1ms/step - loss: 1.6579 - accuracy: 0.3898 - val_loss: 1.5336 - val_accuracy: 0.4414
Epoch 3/250
50000/50000 [==============================] - 73s 1ms/step - loss: 1.5074 - accuracy: 0.4500 - val_loss: 1.4007 - val_accuracy: 0.4904
Epoch 4/250
50000/50000 [==============================] - 73s 1ms/step - loss: 1.4134 - accuracy: 0.4867 - val_loss: 1.3127 - val_accuracy: 0.5330
Epoch 5/250
50000/50000 [==============================] - 73s 1ms/step - loss: 1.3452 - accuracy: 0.5153 - val_loss: 1.2466 - val_accuracy: 0.5533
Epoch 6/250
50000/50000 [==============================] - 73s 1ms/step - loss: 1.2784 - accuracy: 0.5422 - val_loss: 1.1808 - val_accuracy: 0.5835
Epoch 7/250
50000/50000 [==============================] - 73s

In [7]:
def save_model_to_db(model, client, db, dbconnection, model_name):
    import pickle
    import time
    import pymongo
    #pickling the model
    pickled_model = pickle.dumps(model)
    
    #saving model to mongoDB
    # creating connection
    myclient = pymongo.MongoClient(client)
    
    #creating database in mongodb
    mydb = myclient[db]
    
    #creating collection
    mycon = mydb[dbconnection]
    info = mycon.insert_one({model_name: pickled_model, 'name': model_name, 'created_time':time.time()})
    print(info.inserted_id, ' saved with this id successfully!')
    
    details = {
        'inserted_id':info.inserted_id,
        'model_name':model_name,
        'created_time':time.time()
    }
    print(details)
    
    return details

In [8]:
import pymongo
def load_saved_model_from_db(model_name, client, db, dbconnection):
    json_data = {}
    
    #saving model to mongoDB
    # creating connection
    myclient = pymongo.MongoClient(client)
    
    #creating database in mongodb
    mydb = myclient[db]
    
    #creating collection
    mycon = mydb[dbconnection]
    data = mycon.find({'name': model_name})
    
    
    for i in data:
        json_data = i
    #fetching model from db
    pickled_model = json_data[model_name]
    
    return pickle.loads(pickled_model)

In [9]:
details = save_model_to_db(model = xgb, client ='mongodb://localhost:27017/', db = 'mydatabase', 
                 dbconnection = 'customers', model_name = 'myxgb')

5eaabe9bbfdb60940486afc8  saved with this id successfully!
{'inserted_id': ObjectId('5eaabe9bbfdb60940486afc8'), 'model_name': 'myxgb', 'created_time': 1588248219.1888342}


In [10]:
xgb  = load_saved_model_from_db(model_name = details['model_name'], client = 'mongodb://localhost:27017/', 
                         db = 'mydatabase', dbconnection = 'customers')

<IPython.core.display.Javascript object>

In [11]:


scores = xgb.evaluate(X_test / 255.0, to_categorical(Y_test))

10000/10000 [==============================] - 3s 292us/step


In [12]:
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

Loss: 0.643
Accuracy: 0.779


In [14]:
import tensorflow as tf

from keras.models import load_model

xgb.save('model_vgg19.h5')
